# Statevector snapshot comparison

Statevector snapshots are a technique that enables the reduction of simulation time from quadratic to linear for certain QBM algorithms.
Typically, simulating $n$ steps of a QBM requires $\frac{n(n+1)}{2}$ time step circuits, and the longest circuit is exactly $n$ time steps long.

Using the statevector sampling method, `qlbm` reduces this to just $n$ total time step simulations by passing read-only copies of the statevector between different components. With this method, a single time step circuit needs to be compiled, which can be reused for all $n$ iterations.

Note that since snapshtos requrie a full and accurate copy of the underlying statevector, this techinque is only feasible on simulators. Within `qlbm`, statevector snapshots can be enabled by toggling the `statevector_snapshots` parameter of the `run` method of a `Runner`. More generally, statevector snapshots are built into `qlbm`'s _reinitialization_ interface, which occurs at the transition between timesteps of QBM algorithms.

In this notebook, we showcase this feature and analyse its impact.

In [ ]:
%pip install qlbm matplotlib seaborn pandas

In [ ]:
import logging.config
from logging import Logger, getLogger

from qiskit_aer import AerSimulator

from qlbm.components import (
    CQLBM,
    CollisionlessInitialConditions,
    EmptyPrimitive,
    GridMeasurement,
)
from qlbm.infra import QiskitRunner, SimulationConfig
from qlbm.lattice import CollisionlessLattice
from qlbm.tools.utils import create_directory_and_parents, get_circuit_properties

In [ ]:
# Import analysis and plotting utilities
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

sns.set_theme()

## Step 1: Setup

Before benchmarking the simulations, we first have to choose some behaviour to simulate and parameters for our quantum circuits.

In [ ]:
def benchmark(
    lattice_dicts,
    num_shots: int,
    num_steps: int,
    statevector_snapshots: bool,
    statevector_sampling: bool,
    logger: Logger,
    dummy_logger: Logger,
    root_output_dir: str,
    target_platform="QISKIT",
    compiler_platform="QISKIT",
    execution_backend=AerSimulator(method="statevector"),
    sampling_backend=AerSimulator(method="statevector"),
    num_repetitions: int = 5,
) -> None:
    for rep in range(num_repetitions):
        logger.info(f"Repetition #{rep} of {num_repetitions}")
        for count, lattice_dict in enumerate(lattice_dicts):
            logger.info(f"Combination #{count + 1} of {len(lattice_dicts)}")

            lattice = CollisionlessLattice(lattice_dict, logger=dummy_logger)
            logger.info(
                f"Lattice={lattice.logger_name()}, num_qubits={lattice.num_total_qubits}"
            )

            output_dir = f"{root_output_dir}/qiskit-{lattice.logger_name()}-{statevector_snapshots}"

            create_directory_and_parents(output_dir)

            cfg = SimulationConfig(
                initial_conditions=CollisionlessInitialConditions(
                    lattice, logger=dummy_logger
                ),
                algorithm=CQLBM(lattice, logger=dummy_logger),
                postprocessing=EmptyPrimitive(lattice, logger=dummy_logger),
                measurement=GridMeasurement(lattice, logger=dummy_logger),
                target_platform=target_platform,
                compiler_platform=compiler_platform,
                optimization_level=0,
                statevector_sampling=statevector_sampling,
                execution_backend=execution_backend,
                sampling_backend=sampling_backend,
                logger=dummy_logger,
            )

            cfg.prepare_for_simulation()

            logger.info(
                f"Final circuit properties: {get_circuit_properties(cfg.algorithm)}"
            )
            # Create a runner object to simulate the circuit
            runner = QiskitRunner(cfg, lattice, logger=logger)

            # Simulate the circuits using both snapshots and sampling
            runner.run(
                num_steps,  # Number of time steps
                num_shots,  # Number of shots per time step
                output_dir,
                statevector_snapshots=statevector_snapshots,
            )

In [ ]:
!mkdir -p qlbm-output/benchmark-statevector-snapshots/ && touch qlbm-output/qiskit-qulacs-comparison/qlbm.log
!:> qlbm-output/benchmark-statevector-snapshots

In [ ]:
NUM_SHOTS = 2**12
NUM_STEPS = 4
NUM_REPS = 3
ROOT_OUTPUT_DIR = "qlbm-output/benchmark-statevector-snapshots"

In [ ]:
create_directory_and_parents(ROOT_OUTPUT_DIR)

lattices = [
    {
        "lattice": {"dim": {"x": 8, "y": 8}, "velocities": {"x": 4, "y": 4}},
        "geometry": [],
    },
    {
        "lattice": {"dim": {"x": 8, "y": 8}, "velocities": {"x": 4, "y": 4}},
        "geometry": [{"x": [5, 6], "y": [1, 2], "boundary": "specular"}],
    },
    {
        "lattice": {"dim": {"x": 8, "y": 8}, "velocities": {"x": 4, "y": 4}},
        "geometry": [
            {"x": [5, 6], "y": [1, 2], "boundary": "specular"},
            {"x": [5, 6], "y": [5, 6], "boundary": "specular"},
        ],
    },
]

dummy_logger = getLogger("dummy")

# By logging at this point we ignore the output of circuit creation
logging.config.fileConfig("statevector_snapshots_logging.conf")
logger = getLogger("qlbm")

## Step 2: Simulation

Now that we have setup our simulations and configered logging, we can simply run the simulations by calling the `benchmark` function!

> **_CAUTION:_** Running the cells below will probably take a few minutes. Each cell should be run exactly once.

In [ ]:
logger.info("Session: snapshots=True")
benchmark(
    lattices,
    NUM_SHOTS,
    NUM_STEPS,
    True,
    True,
    logger,
    dummy_logger,
    ROOT_OUTPUT_DIR,
    num_repetitions=NUM_REPS,
)

In [ ]:
logger.info("Session: snapshots=False")
benchmark(
    lattices,
    NUM_SHOTS,
    NUM_STEPS,
    False,
    False,
    logger,
    dummy_logger,
    ROOT_OUTPUT_DIR,
    num_repetitions=NUM_REPS,
)

## Step 3: Analysis

Once the simulation has concluded, the performance logs created by `qlbm` will be under `qlbm-output/benchmark-statevector-snapshots/qlbm.log`.
The scripts below will parse this file, extract useful information, format it, and plot it for convenient analysis.

In [ ]:
log_file = "qlbm-output/benchmark-statevector-snapshots/qlbm.log"
with open(log_file, "r") as f:
    lines = f.readlines()

session_line = [c for c, line in enumerate(lines) if "Session" in line][1]

lines_statevector_true = lines[:session_line]
lines_statevector_false = lines[session_line:]

In [ ]:
# Process statevector=True lines
combination_lines_indices = [
    c for c, line in enumerate(lines_statevector_true) if "Combination #" in line
]

sections = []
for c in range(len((combination_lines_indices))):
    if c < len(combination_lines_indices) - 1:
        sections.append(
            lines_statevector_true[
                combination_lines_indices[c] : combination_lines_indices[c + 1]
            ]
        )
    else:
        sections.append(lines_statevector_true[combination_lines_indices[c] :])

statevector_true_records = []
for section in sections:
    sec_info = section[1].split("INFO: ")[-1].rstrip().split(", ")
    time_elapsed_ns = section[-1].split("INFO: ")[-1].rstrip().split()[-2]
    step_simulation_line_indices = [
        c for c, line in enumerate(section) if "Main circuit for step" in line
    ]
    total_duration = 0
    for sl in step_simulation_line_indices:
        step_number = section[sl].split("for step ")[-1].split()[0]
        props = section[sl].split("INFO: ")[-1].rstrip().split(", ")[1:]
        duration = section[sl + 1].split()[-2]
        total_duration += int(duration)
        statevector_true_records.append(
            {
                "Lattice": sec_info[0].split("=")[-1].split("/")[-1].split(".")[0],
                "Dimensions": sec_info[0].split("=")[-1].split("-")[1],
                "Obstacles": int(sec_info[0].split("=")[-1].split("-")[-2]),
                "Circuit Qubits": int(sec_info[1].split("=")[-1]),
                "Step": int(step_number),
                "Depth": int(props[1]),
                "Gates": int(props[-1][:-1]),
                "Duration (ns)": int(duration),
                "Cumulative Duration (ns)": int(total_duration),
                "Snapshots": True,
            }
        )

sv_true_df = pd.DataFrame.from_records(statevector_true_records)
sv_true_df

In [ ]:
# Process statevector=False lines
combination_lines_indices = [
    c for c, line in enumerate(lines_statevector_false) if "Combination #" in line
]

sections = []
for c in range(len((combination_lines_indices))):
    if c < len(combination_lines_indices) - 1:
        sections.append(
            lines_statevector_false[
                combination_lines_indices[c] : combination_lines_indices[c + 1]
            ]
        )
    else:
        sections.append(lines_statevector_false[combination_lines_indices[c] :])

statevector_false_records = []
for section in sections:
    sec_info = section[1].split("INFO: ")[-1].rstrip().split(", ")
    time_elapsed_ns = section[-1].split("INFO: ")[-1].rstrip().split()[-2]
    step_simulation_line_indices = [
        c for c, line in enumerate(section) if "Main circuit for step" in line
    ]
    total_duration = 0
    for sl in step_simulation_line_indices:
        step_number = section[sl].split("for step ")[-1].split()[0]
        props = section[sl].split("INFO: ")[-1].rstrip().split(", ")[1:]
        duration = section[sl + 1].split()[-2]

        total_duration += int(duration)
        statevector_false_records.append(
            {
                "Lattice": sec_info[0].split("=")[-1].split("/")[-1].split(".")[0],
                "Dimensions": sec_info[0].split("=")[-1].split("-")[1],
                "Obstacles": int(sec_info[0].split("=")[-1].split("-")[-2]),
                "Circuit Qubits": int(sec_info[1].split("=")[-1]),
                "Step": int(step_number),
                "Depth": int(props[1]),
                "Gates": int(props[-1][:-1]),
                "Duration (ns)": int(duration),
                "Cumulative Duration (ns)": int(total_duration),
                "Snapshots": False,
            }
        )

sv_false_df = pd.DataFrame.from_records(statevector_false_records)
sv_false_df

In [ ]:
cdf = pd.concat([sv_true_df, sv_false_df])
cdf["Duration (s)"] = cdf["Duration (ns)"] / 1e9
cdf["Cumulative Duration (s)"] = cdf["Cumulative Duration (ns)"] / 1e9

In [ ]:
ax = sns.lineplot(
    cdf,
    x="Step",
    y="Cumulative Duration (s)",
    hue="Obstacles",
    style="Snapshots",
    markers=True,
)
# plt.yscale("log")
plt.xticks(pd.unique(cdf["Step"]))
ax.legend(loc="center left", bbox_to_anchor=(1, 0.5))